#### This notebook evaluates the identity model on the Norman dataset.

The identity model (= "no effect model") assumes that perturbations have no effect and predictions thus correspond to control cells. These 'predictions' are computed on-the-fly in this notebook (rather than being loaded as in the other baseline models).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import jax
import os
from cfp.metrics import compute_metrics, compute_mean_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp
import anndata as ad
import pandas as pd
from tqdm.auto import tqdm

In [3]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]

In [4]:
split = 0

#### Data loading

In [5]:
DATA_DIR = "/home/haicu/soeren.becker/repos/ot_pert_reproducibility/norman2019/norman_preprocessed_adata"

adata_train_path = os.path.join(DATA_DIR, f"adata_train_pca_3_split_{split}.h5ad")
adata_test_path = os.path.join(DATA_DIR, f"adata_val_pca_3_split_{split}.h5ad")
adata_ood_path = os.path.join(DATA_DIR, f"adata_test_pca_3_split_{split}.h5ad")

# load data splits
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

#### Add subgroup annotations

- single: Only one gene has been perturbed. This perturbation is not part of the training data.
- double_seen_0: Two genes have been perturbed. Neither of these gene perturbations have been seen as single perturbations in the training data.
- double_seen_1: Two genes have been perturbed. One of the two perturbed genes has been seen as single perturbation in the training data.
- double_seen_2: Two genes have been perturbed. Both gene perturbations have been seen as single perturbation in the training data.

In [6]:
train_conditions = adata_train.obs.condition.str.replace("+ctrl", "").str.replace("ctrl+", "").unique()

assert not adata_ood[adata_ood.obs.condition != "ctrl"].obs.condition.isin(train_conditions).any()

mask_single_perturbation = adata_ood.obs.condition.str.contains("ctrl")
mask_double_perturbation_seen_0 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    ~adata_ood.obs.gene_1.isin(train_conditions) & 
    ~adata_ood.obs.gene_2.isin(train_conditions)
)
mask_double_perturbation_seen_1 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    (
        (adata_ood.obs.gene_1.isin(train_conditions) & ~adata_ood.obs.gene_2.isin(train_conditions)) | 
        (~adata_ood.obs.gene_1.isin(train_conditions) & adata_ood.obs.gene_2.isin(train_conditions))
    )
)
mask_double_perturbation_seen_2 = (
    ~adata_ood.obs.condition.str.contains("ctrl") & 
    adata_ood.obs.gene_1.isin(train_conditions) & 
    adata_ood.obs.gene_2.isin(train_conditions)
)
adata_ood.obs.loc[mask_single_perturbation, "subgroup"] = "single"
adata_ood.obs.loc[mask_double_perturbation_seen_0, "subgroup"] = "double_seen_0"
adata_ood.obs.loc[mask_double_perturbation_seen_1, "subgroup"] = "double_seen_1"
adata_ood.obs.loc[mask_double_perturbation_seen_2, "subgroup"] = "double_seen_2"

display(adata_ood.obs.subgroup.value_counts())

subgroup
double_seen_1    11417
single           11317
double_seen_2     4593
double_seen_0     1927
Name: count, dtype: int64

In [7]:
# compute pca on full dataset
adata_all = ad.concat((adata_train, adata_test, adata_ood))
cfpp.centered_pca(adata_all, n_comps=10)

/home/haicu/soeren.becker/miniconda3/envs/env_cfp2/lib/python3.12/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


#### Predict on test set

In [8]:
# compute predictions of identity model --> the identity model always predicts that the perturbation has not effect, 
# i.e., predictions correspond to cells of the control condition
adata_pred_test = adata_test[adata_test.obs["condition"]=="ctrl"]

# project predictions and ground truth data onto pca space
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_all)
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_all)

# dict for ground truths per condition, encoded (=pca space) and decoded (= gene space)
test_data_target_encoded, test_data_target_decoded = {}, {}

# dict for predictions per condition, encoded (=pca space) and decoded (= gene space)
test_data_target_encoded_predicted, test_data_target_decoded_predicted = {}, {}

for cond in adata_test.obs["condition"].cat.categories:
    if cond == "ctrl":
        continue

    # pca space
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_encoded_predicted[cond] = adata_pred_test.obsm["X_pca"]
    
    # gene space
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X
    test_data_target_decoded_predicted[cond] = adata_pred_test.X

/ictstr01/home/haicu/soeren.becker/repos/cell_flow_perturbation/src/cfp/preprocessing/_pca.py:192: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  query_adata.obsm[obsm_key_added] = np.array(


#### Predict on ood set (full ood set + subgroups)

In [9]:
# compute predictions of identity model --> the identity model always predicts that the perturbation has not effect, 
# i.e., predictions correspond to cells of the control condition
adata_pred_ood = adata_ood[adata_ood.obs["condition"]=="ctrl"]

cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_all)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_all)

ood_data_target_encoded, ood_data_target_decoded = {}, {}
ood_data_target_encoded_predicted, ood_data_target_decoded_predicted = {}, {}

subgroups = ["all", "single", "double_seen_0", "double_seen_1", "double_seen_0"]

for subgroup in subgroups:

    ood_data_target_encoded[subgroup] = {}
    ood_data_target_decoded[subgroup] = {}
    ood_data_target_encoded_predicted[subgroup] = {}
    ood_data_target_decoded_predicted[subgroup] = {}
    
    for cond in adata_ood.obs["condition"].cat.categories:
        if cond == "ctrl":
            continue
        
        select = adata_ood.obs["condition"] == cond
        if subgroup != "all":
            select = select & (adata_ood.obs.subgroup == subgroup)

        if not any(select):
            # the condition is not part of this subgroup
            continue
        
        # pca space
        ood_data_target_encoded[subgroup][cond] = adata_ood[select].obsm["X_pca"]
        ood_data_target_encoded_predicted[subgroup][cond] = adata_pred_ood.obsm["X_pca"]
    
        # gene space
        ood_data_target_decoded[subgroup][cond] = adata_ood[select].X
        ood_data_target_decoded_predicted[subgroup][cond] = adata_pred_ood.X

#### Evaluation on test set

In [19]:
# test set: evaluation in encoded (=pca) space
test_metrics_encoded = jax.tree_util.tree_map(
    # compute_metrics, 
    compute_metrics_fast, 
    test_data_target_encoded, 
    test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(
    test_metrics_encoded, 
    prefix="encoded_test_"
)

# test set: evaluation in decoded (=gene) space
test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, 
    test_data_target_decoded, 
    test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(
    test_metrics_decoded, 
    prefix="decoded_test_"
)

# test set
test_deg_dict = {
    k: v 
    for k, v in adata_train.uns['rank_genes_groups_cov_all'].items() 
    if k in test_data_target_decoded_predicted.keys()
}

# test set
test_deg_target_decoded_predicted = jax.tree_util.tree_map(
    get_mask, 
    test_data_target_decoded_predicted, 
    test_deg_dict
)
test_deg_target_decoded = jax.tree_util.tree_map(
    get_mask, 
    test_data_target_decoded, 
    test_deg_dict
)

deg_test_metrics = jax.tree_util.tree_map(
    # compute_metrics, 
    compute_metrics_fast, 
    test_deg_target_decoded, 
    test_deg_target_decoded_predicted
)
deg_mean_test_metrics = compute_mean_metrics(
    deg_test_metrics, 
    prefix="deg_test_"
)

In [20]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -0.9467595931022398,
 'encoded_test_e_distance': 21.113649284174276,
 'encoded_test_mmd_distance': 0.05591186839005639}

In [21]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.9612018273722741,
 'decoded_test_e_distance': 29.58447520908832,
 'decoded_test_mmd_distance': 0.025993242604477752}

In [22]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.807751226809717,
 'deg_test_e_distance': 22.63092522060512,
 'deg_test_mmd_distance': 0.06167025122070505}

#### Evaluation on ood set

In [23]:
ood_metrics_encoded, mean_ood_metrics_encoded = {}, {}
ood_metrics_decoded, mean_ood_metrics_decoded = {}, {}
deg_ood_metrics, deg_mean_ood_metrics = {}, {}
ood_deg_dict = {}
ood_deg_target_decoded_predicted, ood_deg_target_decoded = {}, {}

for subgroup in tqdm(subgroups[::-1]):

    print(f"subgroup: {subgroup}")

    print("Computing ood_metrics_encoded")
    # ood set: evaluation in encoded (=pca) space
    ood_metrics_encoded[subgroup] = jax.tree_util.tree_map(
        # compute_metrics, 
        compute_metrics_fast, 
        ood_data_target_encoded[subgroup], 
        ood_data_target_encoded_predicted[subgroup]
    )
    mean_ood_metrics_encoded[subgroup] = compute_mean_metrics(
        ood_metrics_encoded[subgroup], 
        prefix="encoded_ood_",
    )

    print("Computing ood_metrics_decoded")
    # ood set: evaluation in decoded (=gene) space
    ood_metrics_decoded[subgroup] = jax.tree_util.tree_map(
        compute_metrics_fast, 
        ood_data_target_decoded[subgroup], 
        ood_data_target_decoded_predicted[subgroup]
    )
    mean_ood_metrics_decoded[subgroup] = compute_mean_metrics(
        ood_metrics_decoded[subgroup], 
        prefix="decoded_ood_",
    )

    # ood set
    ood_deg_dict[subgroup] = {
        k: v
        for k, v in adata_train.uns['rank_genes_groups_cov_all'].items() 
        if k in ood_data_target_decoded_predicted[subgroup].keys()
    }

    print("Apply DEG mask")
    # ood set
    ood_deg_target_decoded_predicted[subgroup] = jax.tree_util.tree_map(
        get_mask, 
        ood_data_target_decoded_predicted[subgroup], 
        ood_deg_dict[subgroup]
    )
    
    ood_deg_target_decoded[subgroup] = jax.tree_util.tree_map(
        get_mask, 
        ood_data_target_decoded[subgroup], 
        ood_deg_dict[subgroup]
    )

    print("Compute metrics on DEG subsetted decoded")
    deg_ood_metrics[subgroup] = jax.tree_util.tree_map(
        # compute_metrics, 
        compute_metrics_fast, 
        ood_deg_target_decoded[subgroup], 
        ood_deg_target_decoded_predicted[subgroup]
    )
    deg_mean_ood_metrics[subgroup] = compute_mean_metrics(
        deg_ood_metrics[subgroup], 
        prefix="deg_ood_"
    )

  0%|          | 0/5 [00:00<?, ?it/s]

subgroup: double_seen_0
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: double_seen_1
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: double_seen_0
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: single
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded
subgroup: all
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded


In [24]:
mean_ood_metrics_encoded

{'double_seen_0': {'encoded_ood_r_squared': -1.0625860095024109,
  'encoded_ood_e_distance': 41.34483749488851,
  'encoded_ood_mmd_distance': 0.09608994983136654},
 'double_seen_1': {'encoded_ood_r_squared': -0.9417627128687772,
  'encoded_ood_e_distance': 45.93255848524384,
  'encoded_ood_mmd_distance': 0.09454133274795656},
 'single': {'encoded_ood_r_squared': -1.4774351550473108,
  'encoded_ood_e_distance': 17.455374982996112,
  'encoded_ood_mmd_distance': 0.04616158101190296},
 'all': {'encoded_ood_r_squared': -1.1948339671732109,
  'encoded_ood_e_distance': 34.74240573665601,
  'encoded_ood_mmd_distance': 0.07708435718004948}}

In [25]:
mean_ood_metrics_decoded

{'double_seen_0': {'decoded_ood_r_squared': 0.9290193617343903,
  'decoded_ood_e_distance': 51.601263640622,
  'decoded_ood_mmd_distance': 0.04310204011077682},
 'double_seen_1': {'decoded_ood_r_squared': 0.9226068596948277,
  'decoded_ood_e_distance': 56.05413620752554,
  'decoded_ood_mmd_distance': 0.04034265025984496},
 'single': {'decoded_ood_r_squared': 0.9670263926188151,
  'decoded_ood_e_distance': 24.169279849000336,
  'decoded_ood_mmd_distance': 0.021967441535606567},
 'all': {'decoded_ood_r_squared': 0.9400034396447868,
  'decoded_ood_e_distance': 43.66084116326818,
  'decoded_ood_mmd_distance': 0.03406024058610549}}

In [26]:
deg_mean_ood_metrics

{'double_seen_0': {'deg_ood_r_squared': 0.6453550110260645,
  'deg_ood_e_distance': 31.879554067707645,
  'deg_ood_mmd_distance': 0.09031646667669217},
 'double_seen_1': {'deg_ood_r_squared': 0.7215764969587326,
  'deg_ood_e_distance': 33.654820371900875,
  'deg_ood_mmd_distance': 0.08941872015764768},
 'single': {'deg_ood_r_squared': 0.859203490946028,
  'deg_ood_e_distance': 17.035811578257867,
  'deg_ood_mmd_distance': 0.0478642490795917},
 'all': {'deg_ood_r_squared': 0.7561317554144101,
  'deg_ood_e_distance': 27.973772498887364,
  'deg_ood_mmd_distance': 0.07402312134122738}}

In [29]:
collected_results = {
    # test
    "test_metrics_encoded": test_metrics_encoded,
    "mean_test_metrics_encoded": mean_test_metrics_encoded,
    "test_metrics_decoded": test_metrics_decoded, 
    "mean_test_metrics_decoded": mean_test_metrics_decoded,
    "test_deg_dict": test_deg_dict,
    "test_deg_target_decoded_predicted": test_deg_target_decoded_predicted,
    "test_deg_target_decoded": test_deg_target_decoded,
    "deg_test_metrics": deg_test_metrics,
    "deg_mean_test_metrics": deg_mean_test_metrics,
    # ood
    "ood_metrics_encoded": ood_metrics_encoded,
    "mean_ood_metrics_encoded": mean_ood_metrics_encoded,
    "ood_metrics_decoded": ood_metrics_decoded,
    "mean_ood_metrics_decoded": mean_ood_metrics_decoded,
    "deg_ood_metrics": deg_ood_metrics,
    "deg_mean_ood_metrics": deg_mean_ood_metrics,
    "ood_deg_dict": ood_deg_dict,
    "ood_deg_target_decoded_predicted": ood_deg_target_decoded_predicted,
    "ood_deg_target_decoded": ood_deg_target_decoded,
}

In [31]:
pd.to_pickle(collected_results, f"./norman_identity_split_{split}_collected_results.pkl")